# Introduction to Molecular Dynamics Simulations with AMBER

-**Author:** Bruno Di Geronimo  
-**Contact:** bgeronimo3@gatech.edu  
-**Github:** https://github.com/BruDiGe/biocatcodeexpander-2024.git  

## Useful links:
1 [AMBER](https://ambermd.org/index.php)


2 [H++](http://newbiophysics.cs.vt.edu/H++/hppdetails.php)


3 [pdb4amber](https://ambermd.org/tutorials/basic/tutorial9/index.php)


4 [LEaP](https://ambermd.org/tutorials/pengfei/)

5 [PyMOL](https://pymolwiki.org/index.php/Main_Page)

6 [Advance AMBER tutorial](https://ambermd.org/tutorials/advanced/tutorial41/index.php#3.%20tICA%20Analysis%20and%20MSM%20Construction)


## Index of Actions:

### 1. Instal Ambertools & parmed using CONDA

First we will install all the software we need for this tutorial.

This could take about 5 mins.


### 2. Create your PDBs
Using PyMol we are going to build our COSTB2 models. Using ChainA from PDB-id **6GGI** and substrate (geranyl geranyl phosphate or **GGS**) from PDB-id **5GUE**.

**ACHTUNG!:** GGS is in *tio* state and without hydrogens. Use builder to modify the substrate.


Use [H++](http://newbiophysics.cs.vt.edu/H++/hppdetails.php) webserver to get protonation state of your 6GGI_Chain-A. Only use aa.


At the end we should have next structures:
* 6GGI-A_H.pdb (protein)
* GSSH-A.pdb (substrate)
* WAT-A.pdb (water molecules, yes cristallographic waters are important)
* MG-A.pdb (magnesium ion)

*Apo system* (**6GGI-A**) without substrate.
*Holo system* (**6GGI-A-GSS**) with substrate.


### 3. tleap
Using tleap we can build our corresponding topology and initial coordinates using the corresponding PDBs and force fields (ff19SB and TIP3P water molecules).


### 4. Analysis with Parmed
Use parmed to visualize and understand how it looks our topology file inside.


### 5. Conventional MD
In this course, we will not run cMD due to the lack of resources & time. Take a look to the scheme proposed and inputs at (Link-Github).

### 6. Visualization of the Molecular Dynamic simulations
Download from [link] files:

1. **APO:** 6GGI-A_equi.pdb and 6GGI-A-md_500ns.dcd


2. **HOLO:** 6GGI-A-GSS_equi.pdb and 6GGI-A-GSS-md_500ns.dcd

Use PyMol to load the trajectories. For **APO** open first the PDB file *6GGI-A_equi.pdb* and the at console enter (load_traj 6GGI-A-md_500ns.dcd). For **HOLO**, in a new PyMol window open first the PDB file *6GGI-A-GSS_equi.pdb* and the at console enter (load_traj 6GGI-A-md_500ns.dcd).


### 7. Trajectory analysis with CPPTRAJ:

Using *cpptraj* software we will analyze both trajectories and plot the corresponding RMSD, RMSF and LIE values.


### 8. Discussion and conclusions

# 1. Instal Ambertools & parmed using CONDA

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda --version

In [ ]:
!conda create --name AmberTools23

In [ ]:
!source activate base

In [ ]:
!conda install -c conda-forge ambertools=23

In [ ]:
!source activate AmberTools23

In [ ]:
!which cpptraj

In [ ]:
!which tleap

In [ ]:
!pip install parmed

In [ ]:
!which parmed

# 2. Create your PDBs

Follow instructor order to obtain the corresponding PDBs:

* **6GGI-A_H.pdb** (protein)
* **GSSH-A.pdb** (substrate)
* **WAT-A.pdb** (water molecules, yes cristallographic waters are important)
* **MG-A.pdb** (magnesium ion)

Files can be also downloaded here: [link](https://github.com/BruDiGe/tutorial-MD/tree/main/00_structures)

# 3. tleap

Tleap works by transforming our PDBs in the corresponding topology file (that has all the information related to atom types, residues, bonds, angles, dihedrals, etc) and the initial coordinates (just x,y,z coordinates).



![AMBER Diagram](https://ambermd.org/tutorials/basic/tutorial9/include/amber_flow.png)


<div class="alert alert-block alert-info">
<b>NOTE</b> Use blue boxes for Tips and notes.
</div>

# 4. Analysis with Parmed

# 5. Conventional MD

# 6. Visualization of the Molecular Dynamic simulations

# 7. Trajectory analysis with CPPTRAJ:

# 8. Discussion and conclusions